In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
from keras import applications
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
import time
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

batch_size = 16
num_classes = 10
epochs = 5
data_augmentation = False
num_predictions = 20

Using TensorFlow backend.


# Parse do dataset e separação entre conjuntos de teste e treinamento

In [2]:
# auxiliary function
def depth_two_uint8_to_float(top_bits, bottom_bits):
    """ Converts a RGB-coded depth into float valued depth. """
    depth_map = (top_bits * 2**8 + bottom_bits).astype('float32')
    depth_map /= float(2**16 - 1)
    depth_map *= 5.0
    return depth_map

# load annotations of this set
with open(os.path.join(set, 'anno_%s.pickle' % set), 'rb') as fi:
    anno_all = pickle.load(fi)

X = []
y = []
# iterate samples of the set
for sample_id, anno in anno_all.items():
    # load data
    image = cv2.imread(os.path.join(set, 'color', '%.5d.png' % sample_id))
    mask = cv2.imread(os.path.join(set, 'mask', '%.5d.png' % sample_id))
    mask = cv2.normalize(mask, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    X.append(image)
    y.append(mask)

    
X = np.array(X)
y = np.array(y)

print(X.shape)
print(y.shape)

TypeError: expected str, bytes or os.PathLike object, not type

In [7]:
img_height = img_width = 32

# Instanciando o modelo Resnet50V2
base_model = applications.resnet_v2.ResNet50V2(weights=None, include_top=False, input_shape=(img_height, img_width, 3))

# Aplicando pooling e dropout no output
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)

# Gerando camada de output e instanciando o modelo
predictions = Dense(num_classes, activation='softmax')(x)
model2 = Model(inputs=base_model.input, outputs=predictions)

from keras.optimizers import SGD, Adam

model2.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
start_time = time.time()

model2.fit(x_train, y_train,
      batch_size=batch_size,
      epochs=epochs,
      validation_data=(x_test, y_test),
      shuffle=True)
print("Fitting duration: " + str(time.time() - start_time))

acc = model2.evaluate(x_test, y_test)
print("Test accuracy:" + str(acc[1]))
print("Loss:" + str(acc[0]))

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 366s 7ms/step - loss: 2.3828 - accuracy: 0.2712 - val_loss: 1.7033 - val_accuracy: 0.4109
Epoch 2/5
50000/50000 [==============================] - 366s 7ms/step - loss: 1.7298 - accuracy: 0.4191 - val_loss: 1.5503 - val_accuracy: 0.4905
Epoch 3/5
50000/50000 [==============================] - 373s 7ms/step - loss: 1.4785 - accuracy: 0.4972 - val_loss: 1.6820 - val_accuracy: 0.4585
Epoch 4/5
50000/50000 [==============================] - 384s 8ms/step - loss: 1.3072 - accuracy: 0.5522 - val_loss: 1.2296 - val_accuracy: 0.5829
Epoch 5/5
50000/50000 [==============================] - 378s 8ms/step - loss: 1.1807 - accuracy: 0.5991 - val_loss: 1.2092 - val_accuracy: 0.6138
Fitting duration: 1876.0506908893585
10000/10000 [==============================] - 10s 963us/step
Test accuracy:0.6137999892234802
Loss:1.2092481250762939


## Mesmo com um tempo de treinamento muito maior e uma estrutura mais complexa, a rede RESNET obteve resultado muito proximo á rede neural criada anteriormente, isso pode ser explicado por:
### A rede RESNET necessita de mais tempo de treinamento, devido a quantidade muito maior de camadas.
### Possivelmente essa acurácia melhoraria com mais épocas em seu treinamento.

In [ ]:
model2.metrics_names

NameError: name 'model2' is not defined